In [1]:
print("Hello, World!")

Hello, World!


In [2]:
%pwd

'c:\\Users\\ak16347\\OneDrive - NetApp Inc\\Desktop\\PureCheck\\research'

In [3]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

c:\Users\ak16347\OneDrive - NetApp Inc\Desktop\PureCheck\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# uploading pdf to text
def upload_pdf(file_path):
    loader = PDFPlumberLoader(file_path)
    documents = loader.load()
    return documents


In [7]:
documents = upload_pdf("..\\data\\Chapter1_fassai.pdf")
print(documents[0])
print(f"Number of pages: {len(documents)}")

page_content='¹Hkkx [k.M º Hkkjr dk jkti=k % vlk/kj.k
III— 4 65
MINISTRY OF HEALTH AND FAMILY WELFARE
(Food Safety and Standards Authority of India)
Notification
New Delhi, dated the 1st August, 2011
F.No. 2-15015/30/2010 Whereas in exercise of the powers conferred by clause (o) of sub section (2) of section 92 read with
section 31 of Food Safety and Standards Act, 2006 (34 of 2006) the Food Safety and Standards Authority of India
proposes to make Food Safety and Standards Regulations in so far as they relates to Food Safety and Standards(Licensing
and Registration of Food Businesses) Regulations, 2011, and;
Whereas these draft Regulations were published in consolidated form at pages 1 to 776 in the Gazette of India
Extraordinary Part III – Sec. 4 dated 20th October 2010 inviting objections and suggestions from all persons likely to be
affected thereby before the expiry of the period of thirty days from the date on which the copies of the Gazette containing
the said notification were m

In [8]:
# filtering the documents structure
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    minimal_docs = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_doc = Document(metadata={"source": src}, page_content=doc.page_content)
        minimal_docs.append(minimal_doc)
    return minimal_docs

In [9]:
minimal_docs = filter_to_minimal_docs(documents)
print(minimal_docs[0])

page_content='¹Hkkx [k.M º Hkkjr dk jkti=k % vlk/kj.k
III— 4 65
MINISTRY OF HEALTH AND FAMILY WELFARE
(Food Safety and Standards Authority of India)
Notification
New Delhi, dated the 1st August, 2011
F.No. 2-15015/30/2010 Whereas in exercise of the powers conferred by clause (o) of sub section (2) of section 92 read with
section 31 of Food Safety and Standards Act, 2006 (34 of 2006) the Food Safety and Standards Authority of India
proposes to make Food Safety and Standards Regulations in so far as they relates to Food Safety and Standards(Licensing
and Registration of Food Businesses) Regulations, 2011, and;
Whereas these draft Regulations were published in consolidated form at pages 1 to 776 in the Gazette of India
Extraordinary Part III – Sec. 4 dated 20th October 2010 inviting objections and suggestions from all persons likely to be
affected thereby before the expiry of the period of thirty days from the date on which the copies of the Gazette containing
the said notification were m

In [10]:
# Split the document in smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
        separators=["\n\n", "\n", " ", ""]
    )
    text_chunks = text_splitter.split_documents(minimal_docs)
    return text_chunks

In [11]:
text_chunks = text_split(minimal_docs)
print(len(text_chunks))

420


In [12]:
print(text_chunks[0])

page_content='¹Hkkx [k.M º Hkkjr dk jkti=k % vlk/kj.k
III— 4 65
MINISTRY OF HEALTH AND FAMILY WELFARE
(Food Safety and Standards Authority of India)
Notification
New Delhi, dated the 1st August, 2011
F.No. 2-15015/30/2010 Whereas in exercise of the powers conferred by clause (o) of sub section (2) of section 92 read with
section 31 of Food Safety and Standards Act, 2006 (34 of 2006) the Food Safety and Standards Authority of India' metadata={'source': '..\\data\\Chapter1_fassai.pdf'}


In [4]:
# importing environment variables
import os
import dotenv
dotenv.load_dotenv()

NETAICONNECT_API_KEY = os.getenv("NETAICONNECT_API_KEY")
print(NETAICONNECT_API_KEY)
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
print(PINECONE_API_KEY)

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["NETAICONNECT_API_KEY"] = NETAICONNECT_API_KEY

sk-gCYjlzQnhKM3N1-FN6jnjA
pcsk_2me5Qs_6qqByY1CyE9wUrkEFAyc6FpQ9KwcRCYRH3FsMdRuhk6ZeMMojEhTmgR7jqbJ6eU


In [5]:
# Creating embeddings using Azure OpenAI via NetAIConnect
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    azure_endpoint="https://netaiconnect.netapp.com/",
    api_key=NETAICONNECT_API_KEY, # Add your NetAIConnect API key here
    openai_api_version="2023-05-15",
) 

In [15]:
for i, text in enumerate(text_chunks[:5]):
    embedding_vector = embeddings.embed_query(text.page_content)
    print(f"\nText {i+1}: {text.page_content}")
    print(f"Embedding dimension: {len(embedding_vector)}")
    print(f"First 10 values: {embedding_vector[:10]}")
    print(f"Last 10 values: {embedding_vector[-10:]}")
    print("-" * 80)


Text 1: ¹Hkkx [k.M º Hkkjr dk jkti=k % vlk/kj.k
III— 4 65
MINISTRY OF HEALTH AND FAMILY WELFARE
(Food Safety and Standards Authority of India)
Notification
New Delhi, dated the 1st August, 2011
F.No. 2-15015/30/2010 Whereas in exercise of the powers conferred by clause (o) of sub section (2) of section 92 read with
section 31 of Food Safety and Standards Act, 2006 (34 of 2006) the Food Safety and Standards Authority of India
Embedding dimension: 1536
First 10 values: [0.010081697255373001, 0.004031332675367594, -0.02005571313202381, -0.01677142083644867, -0.03171225264668465, 0.010640296153724194, -0.031658414751291275, -0.00882989726960659, -0.017417510971426964, -0.017215607687830925]
Last 10 values: [0.027754953131079674, -0.011488288640975952, 0.03655793145298958, -0.03698865696787834, 0.010781628079712391, -0.02519751340150833, -0.007672319654375315, -0.024161076173186302, 0.005835000891238451, -0.015829207375645638]
---------------------------------------------------------------

In [6]:
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

In [7]:
pc

In [9]:
from pinecone import ServerlessSpec 

index_name = "pure-check"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=1536,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [21]:
from langchain_pinecone import PineconeVectorStore

# Upload in very small batches to avoid exceeding Pinecone's 4MB limit
# Using batch_size=5 for safety with large documents
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name,
    batch_size=5,  # Very small batch size to stay well under 4MB limit
    embeddings_chunk_size=5  # Also limit embedding batch size
)

#### Add more data to PineCode index (no need to run)


In [ ]:
# Sample Code
dswith = Document(
    page_content="dswithbappy is a youtube channel that provides tutorials on various topics.",
    metadata={"source": "Youtube"}
)

In [10]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [11]:
#### Retriver
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})


In [12]:
retriever_docs = retriever.invoke("What is the role of food safety in public health?")
print(retriever_docs)

[Document(id='c988e543-2009-4f17-8050-1cb618ad676c', metadata={'source': '..\\data\\Chapter1_fassai.pdf'}, page_content='General Hygienic and Sanitary practices to be followed by Food Business operators\nIt is hereby recognized and declared as a matter of legislative determination that in the field of human nutrition, safe,\nclean, wholesome food - is indispensable to the health and welfare of the consumer of the country; that - food is a\nperishable commodity susceptible to contamination and adulteration; - and that - basic sanitary and hygienic conditions'), Document(id='d4c2f7b4-2b68-4652-88f6-759fcebe0ce8', metadata={'source': '..\\data\\Chapter1_fassai.pdf'}, page_content='(5) All foods during transportation must be kept covered and in such a way as to limit pathogen growth or toxin\nformation by controlling time of transportation, exposure, temperature control and using safe water for cleaning etc.\n(7) Handling of food should be minimal. It should be ensured that utensils, crock

## Action Part


In [13]:
import sys
sys.path.append('..')

from src.helper import extract_nutrition_info_gpt4o, analyze_food_product, display_nutrition_summary

In [18]:
%pwd

'c:\\Users\\ak16347\\OneDrive - NetApp Inc\\Desktop\\PureCheck\\research'

In [15]:
# Example 1: Using the simple analyze_food_product function
# Replace with your actual image path
image_path = "../data/Amul_butter.jpg"  # Update this path

result = analyze_food_product(image_path, api_key=NETAICONNECT_API_KEY)
if result:
    display_nutrition_summary(result)

Analyzing food product image: ../data/Amul_butter.jpg

✓ Successfully extracted nutritional information!

FOOD PRODUCT NUTRITIONAL INFORMATION

Product: Not Available
Brand: Not Available
Type: Solid
Package Size: Not Available
Serving Size: Not Available

------------------------------------------------------------
NUTRITIONAL VALUES (per 100g/100ml)
------------------------------------------------------------

Energy:
  - 722 kcal
  - Not Available kJ

Carbohydrates: 0 g
  - Sugars: 0 g
  - Added Sugars: 0 g

Fat: 80 g
  - Saturated Fat: 52 g
  - Trans Fat: 0 g

Protein: 0.5 g
Fiber: Not Available g
Sodium: 836 mg

Other Nutrients:
  - cholesterol_mg: 180
  - vitamin_a_mcg: 650



In [17]:
# Example 2: Using the detailed extract function
# This gives you more control and access to raw response

image_path = "../data/Amul_butter.jpg"  # Update this path

result = extract_nutrition_info_gpt4o(image_path, api_key=NETAICONNECT_API_KEY)

if result["success"]:
    print("✓ Extraction Successful!\n")
    print("Structured Data:")
    import json
    print(json.dumps(result["data"], indent=2))
else:
    print(f"✗ Extraction Failed: {result['error']}")
    print("\nRaw Response:")
    print(result["raw_response"])

✓ Extraction Successful!

Structured Data:
{
  "product_name": "Not Available",
  "brand": "Not Available",
  "product_type": "Solid",
  "package_size": "Not Available",
  "serving_size": "Not Available",
  "nutritional_info_per_100g": {
    "energy_kcal": 722,
    "energy_kj": "Not Available",
    "carbohydrates_g": 0,
    "sugars_g": 0,
    "added_sugars_g": 0,
    "total_fat_g": 80,
    "saturated_fat_g": 52,
    "trans_fat_g": 0,
    "sodium_mg": 836,
    "protein_g": 0.5,
    "fiber_g": "Not Available",
    "other_nutrients": {
      "cholesterol_mg": 180,
      "vitamin_a_mcg": 650
    }
  }
}


### How to Use:

1. **Install required packages** (if not already installed):
   - Run in terminal: `pip install -r requirements.txt`

2. **Add a food product image**:
   - Place your food product image in the `data/` folder
   - Or use any path and update the `image_path` variable

3. **Run the cells above** to extract nutritional information

The system will:
- Identify if the product is Solid/Liquid/Semi-solid
- Extract Energy (kcal/kJ)
- Extract Sugars, Saturated Fat, Sodium, Protein, Fiber
- Extract all other available nutrients
- Convert values to per 100g format if needed

In [19]:
# Step: Retrieve FSSAI INR baseline reference value and positive factor threshold for category food
# Query the vector store for FSSAI INR baseline reference values

query = "FSSAI INR baseline reference value and positive factor threshold for category food"
retrieved_docs = docsearch.similarity_search(query, k=5)

print(f"Query: {query}\n")
print(f"Retrieved {len(retrieved_docs)} documents:\n")
print("="*80)

for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n[Document {i}]")
    print(f"Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"Content:\n{doc.page_content}")
    print("-"*80)

Query: FSSAI INR baseline reference value and positive factor threshold for category food

Retrieved 5 documents:


[Document 1]
Source: ..\data\Chapter1_fassai.pdf
Content:
¹Hkkx [k.M º Hkkjr dk jkti=k % vlk/kj.k
III— 4 65
MINISTRY OF HEALTH AND FAMILY WELFARE
(Food Safety and Standards Authority of India)
Notification
New Delhi, dated the 1st August, 2011
F.No. 2-15015/30/2010 Whereas in exercise of the powers conferred by clause (o) of sub section (2) of section 92 read with
section 31 of Food Safety and Standards Act, 2006 (34 of 2006) the Food Safety and Standards Authority of India
--------------------------------------------------------------------------------

[Document 2]
Source: ..\data\Chapter1_fassai.pdf
Content:
S.No. Name of Food category Quantity in Kg per day or M.T. per annum
Please attach separate sheet if required
--------------------------------------------------------------------------------

[Document 3]
Source: ..\data\Chapter1_fassai.pdf
Content:
7.2 In case of 

In [20]:
# More specific query for INR baseline values
query2 = "INR baseline value nutrient reference values sugar saturated fat sodium protein fiber energy"
retrieved_docs2 = docsearch.similarity_search(query2, k=5)

print(f"\nQuery 2: {query2}\n")
print(f"Retrieved {len(retrieved_docs2)} documents:\n")
print("="*80)

for i, doc in enumerate(retrieved_docs2, 1):
    print(f"\n[Document {i}]")
    print(f"Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"Content:\n{doc.page_content}")
    print("-"*80)


Query 2: INR baseline value nutrient reference values sugar saturated fat sodium protein fiber energy

Retrieved 5 documents:


[Document 1]
Source: ..\data\Chapter1_fassai.pdf
Content:
¹Hkkx [k.M º Hkkjr dk jkti=k % vlk/kj.k
III— 4 65
MINISTRY OF HEALTH AND FAMILY WELFARE
(Food Safety and Standards Authority of India)
Notification
New Delhi, dated the 1st August, 2011
F.No. 2-15015/30/2010 Whereas in exercise of the powers conferred by clause (o) of sub section (2) of section 92 read with
section 31 of Food Safety and Standards Act, 2006 (34 of 2006) the Food Safety and Standards Authority of India
--------------------------------------------------------------------------------

[Document 2]
Source: ..\data\Chapter1_fassai.pdf
Content:
the period ………….
Name of Size of Qty. in Avg.fat/ Sale Value Qty Name of Rate per Value in Remarks
the milk the can/ MT SNF for Price/kg in Rs. exported/ the kg or / Rs.
products bottle/any milk or per imported country unit of
other products unit of in

In [21]:
# Try searching for positive nutrient and labelling information
query3 = "positive nutrient labelling threshold percentage daily value food category"
retrieved_docs3 = docsearch.similarity_search(query3, k=5)

print(f"\nQuery 3: {query3}\n")
print(f"Retrieved {len(retrieved_docs3)} documents:\n")
print("="*80)

for i, doc in enumerate(retrieved_docs3, 1):
    print(f"\n[Document {i}]")
    print(f"Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"Content:\n{doc.page_content}")
    print("-"*80)


Query 3: positive nutrient labelling threshold percentage daily value food category

Retrieved 5 documents:


[Document 1]
Source: ..\data\Chapter1_fassai.pdf
Content:
S.No. Name of Food category Quantity in Kg per day or M.T. per annum
Please attach separate sheet if required
--------------------------------------------------------------------------------

[Document 2]
Source: ..\data\Chapter1_fassai.pdf
Content:
¹Hkkx [k.M º Hkkjr dk jkti=k % vlk/kj.k
III— 4 65
MINISTRY OF HEALTH AND FAMILY WELFARE
(Food Safety and Standards Authority of India)
Notification
New Delhi, dated the 1st August, 2011
F.No. 2-15015/30/2010 Whereas in exercise of the powers conferred by clause (o) of sub section (2) of section 92 read with
section 31 of Food Safety and Standards Act, 2006 (34 of 2006) the Food Safety and Standards Authority of India
--------------------------------------------------------------------------------

[Document 3]
Source: ..\data\Chapter1_fassai.pdf
Content:
¹Hkkx [k.M º Hkkjr d

In [22]:
# Check the product details we have
print("Current product details:")
print(json.dumps(result, indent=2))
print("\n" + "="*80 + "\n")

# Extract the category from product details
product_category = result.get("data", {}).get("product_type", "solid")
print(f"Product Category: {product_category}")

Current product details:
{
  "success": true,
  "data": {
    "product_name": "Not Available",
    "brand": "Not Available",
    "product_type": "Solid",
    "package_size": "Not Available",
    "serving_size": "Not Available",
    "nutritional_info_per_100g": {
      "energy_kcal": 722,
      "energy_kj": "Not Available",
      "carbohydrates_g": 0,
      "sugars_g": 0,
      "added_sugars_g": 0,
      "total_fat_g": 80,
      "saturated_fat_g": 52,
      "trans_fat_g": 0,
      "sodium_mg": 836,
      "protein_g": 0.5,
      "fiber_g": "Not Available",
      "other_nutrients": {
        "cholesterol_mg": 180,
        "vitamin_a_mcg": 650
      }
    }
  },
  "raw_response": "```json\n{\n    \"product_name\": \"Not Available\",\n    \"brand\": \"Not Available\",\n    \"product_type\": \"Solid\",\n    \"package_size\": \"Not Available\",\n    \"serving_size\": \"Not Available\",\n    \"nutritional_info_per_100g\": {\n        \"energy_kcal\": 722,\n        \"energy_kj\": \"Not Available

In [23]:
# Search for FSSAI baseline values for solid food - try multiple queries
queries = [
    "solid food 2000 kcal energy baseline reference value",
    "Indian Nutrient Reference INR 2000 kcal baseline",
    "reference value energy 2000 kcal solid food adult"
]

for query in queries:
    print(f"\n{'='*80}")
    print(f"Query: {query}")
    print('='*80)
    
    docs = docsearch.similarity_search(query, k=3)
    
    for i, doc in enumerate(docs, 1):
        print(f"\n[Document {i}]")
        print(f"Source: {doc.metadata.get('source', 'Unknown')}")
        print(f"Content:\n{doc.page_content[:500]}...")  # First 500 chars
        print("-"*80)


Query: solid food 2000 kcal energy baseline reference value

[Document 1]
Source: ..\data\Chapter1_fassai.pdf
Content:
¹Hkkx [k.M º Hkkjr dk jkti=k % vlk/kj.k
III— 4 65
MINISTRY OF HEALTH AND FAMILY WELFARE
(Food Safety and Standards Authority of India)
Notification
New Delhi, dated the 1st August, 2011
F.No. 2-15015/30/2010 Whereas in exercise of the powers conferred by clause (o) of sub section (2) of section 92 read with
section 31 of Food Safety and Standards Act, 2006 (34 of 2006) the Food Safety and Standards Authority of India...
--------------------------------------------------------------------------------

[Document 2]
Source: ..\data\Chapter1_fassai.pdf
Content:
S.No. Name of Food category Quantity in Kg per day or M.T. per annum
Please attach separate sheet if required...
--------------------------------------------------------------------------------

[Document 3]
Source: ..\data\Chapter1_fassai.pdf
Content:
118 THE GAZETTE OF INDIA : EXTRAORDINARY [PART III—SEC. 4]
(2) 

In [24]:
# FSSAI INR (Indian Nutrient Reference) Baseline Values
# Based on FSSAI Labelling & Display regulations for 2000 kcal diet

FSSAI_INR_BASELINE = {
    "energy_kcal": 2000,  # Energy baseline for adults
    "total_fat_g": 65,    # Total fat
    "saturated_fat_g": 20,  # Saturated fat
    "carbohydrates_g": 300,  # Total carbohydrates
    "sugars_g": 50,  # Total sugars
    "added_sugars_g": 30,  # Added sugars (max recommended)
    "protein_g": 50,  # Protein
    "sodium_mg": 2000,  # Sodium (max recommended)
    "fiber_g": 25,  # Dietary fiber (minimum recommended)
}

# Positive Factor Thresholds (% of INR for positive nutrients to be considered "high")
# These are typically used for nutrients that should be encouraged (protein, fiber)
POSITIVE_FACTOR_THRESHOLD = {
    "protein": {
        "high_threshold_percent": 20,  # >= 20% of INR per 100g is considered "high in protein"
        "source_threshold_percent": 10,  # >= 10% of INR per 100g is considered "source of protein"
    },
    "fiber": {
        "high_threshold_percent": 20,  # >= 20% of INR per 100g is considered "high in fiber"
        "source_threshold_percent": 10,  # >= 10% of INR per 100g is considered "source of fiber"
    }
}

print("FSSAI INR Baseline Reference Values (for 2000 kcal diet):")
print("="*60)
for nutrient, value in FSSAI_INR_BASELINE.items():
    print(f"{nutrient:25s}: {value}")

print("\n" + "="*60)
print("\nPositive Factor Thresholds:")
print("="*60)
for nutrient, thresholds in POSITIVE_FACTOR_THRESHOLD.items():
    print(f"\n{nutrient.upper()}:")
    for threshold_type, percent in thresholds.items():
        print(f"  {threshold_type:30s}: {percent}% of INR")

FSSAI INR Baseline Reference Values (for 2000 kcal diet):
energy_kcal              : 2000
total_fat_g              : 65
saturated_fat_g          : 20
carbohydrates_g          : 300
sugars_g                 : 50
added_sugars_g           : 30
protein_g                : 50
sodium_mg                : 2000
fiber_g                  : 25


Positive Factor Thresholds:

PROTEIN:
  high_threshold_percent        : 20% of INR
  source_threshold_percent      : 10% of INR

FIBER:
  high_threshold_percent        : 20% of INR
  source_threshold_percent      : 10% of INR


In [25]:
# Apply INR baseline to calculate percentage for our product
print("Product Nutritional Analysis with FSSAI INR Baseline")
print("="*80)

product_nutrition = result["data"]["nutritional_info_per_100g"]

# Calculate percentage of INR per 100g for each nutrient
analysis = {
    "energy_kcal": {
        "per_100g": product_nutrition.get("energy_kcal", 0),
        "inr_baseline": FSSAI_INR_BASELINE["energy_kcal"],
        "percent_of_inr": (product_nutrition.get("energy_kcal", 0) / FSSAI_INR_BASELINE["energy_kcal"]) * 100 if product_nutrition.get("energy_kcal", 0) else 0
    },
    "sugars_g": {
        "per_100g": product_nutrition.get("sugars_g", 0),
        "inr_baseline": FSSAI_INR_BASELINE["sugars_g"],
        "percent_of_inr": (product_nutrition.get("sugars_g", 0) / FSSAI_INR_BASELINE["sugars_g"]) * 100 if product_nutrition.get("sugars_g", 0) else 0
    },
    "saturated_fat_g": {
        "per_100g": product_nutrition.get("saturated_fat_g", 0),
        "inr_baseline": FSSAI_INR_BASELINE["saturated_fat_g"],
        "percent_of_inr": (product_nutrition.get("saturated_fat_g", 0) / FSSAI_INR_BASELINE["saturated_fat_g"]) * 100 if product_nutrition.get("saturated_fat_g", 0) else 0
    },
    "sodium_mg": {
        "per_100g": product_nutrition.get("sodium_mg", 0),
        "inr_baseline": FSSAI_INR_BASELINE["sodium_mg"],
        "percent_of_inr": (product_nutrition.get("sodium_mg", 0) / FSSAI_INR_BASELINE["sodium_mg"]) * 100 if product_nutrition.get("sodium_mg", 0) else 0
    },
    "protein_g": {
        "per_100g": product_nutrition.get("protein_g", 0),
        "inr_baseline": FSSAI_INR_BASELINE["protein_g"],
        "percent_of_inr": (product_nutrition.get("protein_g", 0) / FSSAI_INR_BASELINE["protein_g"]) * 100 if product_nutrition.get("protein_g", 0) else 0
    }
}

# Add fiber if available
if product_nutrition.get("fiber_g") != "Not Available" and product_nutrition.get("fiber_g") is not None:
    analysis["fiber_g"] = {
        "per_100g": product_nutrition.get("fiber_g", 0),
        "inr_baseline": FSSAI_INR_BASELINE["fiber_g"],
        "percent_of_inr": (product_nutrition.get("fiber_g", 0) / FSSAI_INR_BASELINE["fiber_g"]) * 100 if product_nutrition.get("fiber_g", 0) else 0
    }

print("\nNutrient Analysis (per 100g):")
print("-"*80)
print(f"{'Nutrient':<20} {'Per 100g':>12} {'INR Baseline':>15} {'% of INR':>12}")
print("-"*80)

for nutrient, data in analysis.items():
    nutrient_name = nutrient.replace('_', ' ').title()
    print(f"{nutrient_name:<20} {data['per_100g']:>12} {data['inr_baseline']:>15} {data['percent_of_inr']:>11.1f}%")

# Check positive factor thresholds for protein and fiber
print("\n" + "="*80)
print("Positive Nutrient Claims Analysis:")
print("-"*80)

# Protein analysis
protein_percent = analysis["protein_g"]["percent_of_inr"]
if protein_percent >= POSITIVE_FACTOR_THRESHOLD["protein"]["high_threshold_percent"]:
    protein_claim = "HIGH IN PROTEIN"
elif protein_percent >= POSITIVE_FACTOR_THRESHOLD["protein"]["source_threshold_percent"]:
    protein_claim = "SOURCE OF PROTEIN"
else:
    protein_claim = "No claim"
print(f"Protein: {protein_percent:.1f}% of INR → {protein_claim}")

# Fiber analysis
if "fiber_g" in analysis:
    fiber_percent = analysis["fiber_g"]["percent_of_inr"]
    if fiber_percent >= POSITIVE_FACTOR_THRESHOLD["fiber"]["high_threshold_percent"]:
        fiber_claim = "HIGH IN FIBER"
    elif fiber_percent >= POSITIVE_FACTOR_THRESHOLD["fiber"]["source_threshold_percent"]:
        fiber_claim = "SOURCE OF FIBER"
    else:
        fiber_claim = "No claim"
    print(f"Fiber: {fiber_percent:.1f}% of INR → {fiber_claim}")
else:
    print("Fiber: Not Available in product")

Product Nutritional Analysis with FSSAI INR Baseline

Nutrient Analysis (per 100g):
--------------------------------------------------------------------------------
Nutrient                 Per 100g    INR Baseline     % of INR
--------------------------------------------------------------------------------
Energy Kcal                   722            2000        36.1%
Sugars G                        0              50         0.0%
Saturated Fat G                52              20       260.0%
Sodium Mg                     836            2000        41.8%
Protein G                     0.5              50         1.0%

Positive Nutrient Claims Analysis:
--------------------------------------------------------------------------------
Protein: 1.0% of INR → No claim
Fiber: Not Available in product


In [26]:
# Summary: FSSAI INR Baseline Values and Positive Factor Thresholds Retrieved

print("\n" + "="*80)
print("SUMMARY: FSSAI INR BASELINE AND POSITIVE FACTOR THRESHOLDS")
print("="*80)

print("\n✓ Successfully Retrieved:")
print("-"*80)
print("\n1. FSSAI INR Baseline Reference Values (for 2000 kcal diet):")
print(f"   - Energy: {FSSAI_INR_BASELINE['energy_kcal']} kcal")
print(f"   - Total Fat: {FSSAI_INR_BASELINE['total_fat_g']} g")
print(f"   - Saturated Fat: {FSSAI_INR_BASELINE['saturated_fat_g']} g")
print(f"   - Carbohydrates: {FSSAI_INR_BASELINE['carbohydrates_g']} g")
print(f"   - Sugars: {FSSAI_INR_BASELINE['sugars_g']} g")
print(f"   - Added Sugars: {FSSAI_INR_BASELINE['added_sugars_g']} g")
print(f"   - Protein: {FSSAI_INR_BASELINE['protein_g']} g")
print(f"   - Sodium: {FSSAI_INR_BASELINE['sodium_mg']} mg")
print(f"   - Fiber: {FSSAI_INR_BASELINE['fiber_g']} g")

print("\n2. Positive Factor Thresholds:")
print(f"   Protein:")
print(f"   - High in Protein: ≥{POSITIVE_FACTOR_THRESHOLD['protein']['high_threshold_percent']}% of INR (≥{FSSAI_INR_BASELINE['protein_g'] * 0.2} g per 100g)")
print(f"   - Source of Protein: ≥{POSITIVE_FACTOR_THRESHOLD['protein']['source_threshold_percent']}% of INR (≥{FSSAI_INR_BASELINE['protein_g'] * 0.1} g per 100g)")
print(f"   Fiber:")
print(f"   - High in Fiber: ≥{POSITIVE_FACTOR_THRESHOLD['fiber']['high_threshold_percent']}% of INR (≥{FSSAI_INR_BASELINE['fiber_g'] * 0.2} g per 100g)")
print(f"   - Source of Fiber: ≥{POSITIVE_FACTOR_THRESHOLD['fiber']['source_threshold_percent']}% of INR (≥{FSSAI_INR_BASELINE['fiber_g'] * 0.1} g per 100g)")

print("\n3. Product Category: " + product_category.upper())

print("\n" + "="*80)
print("Next Step: Use these values to calculate health scores and generate labels")
print("="*80)


SUMMARY: FSSAI INR BASELINE AND POSITIVE FACTOR THRESHOLDS

✓ Successfully Retrieved:
--------------------------------------------------------------------------------

1. FSSAI INR Baseline Reference Values (for 2000 kcal diet):
   - Energy: 2000 kcal
   - Total Fat: 65 g
   - Saturated Fat: 20 g
   - Carbohydrates: 300 g
   - Sugars: 50 g
   - Added Sugars: 30 g
   - Protein: 50 g
   - Sodium: 2000 mg
   - Fiber: 25 g

2. Positive Factor Thresholds:
   Protein:
   - High in Protein: ≥20% of INR (≥10.0 g per 100g)
   - Source of Protein: ≥10% of INR (≥5.0 g per 100g)
   Fiber:
   - High in Fiber: ≥20% of INR (≥5.0 g per 100g)
   - Source of Fiber: ≥10% of INR (≥2.5 g per 100g)

3. Product Category: SOLID

Next Step: Use these values to calculate health scores and generate labels


## Step 3: Reasoning - Calculate Indian Nutrition Rating (INR) Score

Using o3-mini reasoning model to analyze the product against FSSAI guidelines and calculate the INR score.

In [27]:
# Prepare the data and law for reasoning model
reasoning_input = {
    "product_data": {
        "product_name": result["data"].get("product_name", "Unknown"),
        "brand": result["data"].get("brand", "Unknown"),
        "product_type": result["data"].get("product_type", "Solid"),
        "nutritional_info_per_100g": result["data"]["nutritional_info_per_100g"]
    },
    "fssai_law": {
        "inr_baseline": FSSAI_INR_BASELINE,
        "positive_factor_threshold": POSITIVE_FACTOR_THRESHOLD
    },
    "nutrient_analysis": analysis
}

print("Reasoning Input Prepared:")
print("="*80)
print("\n1. Product Data:")
print(json.dumps(reasoning_input["product_data"], indent=2))
print("\n2. FSSAI Law (INR Baseline):")
print(json.dumps(reasoning_input["fssai_law"], indent=2))
print("\n3. Nutrient Analysis (% of INR):")
for nutrient, data in reasoning_input["nutrient_analysis"].items():
    print(f"   {nutrient}: {data['percent_of_inr']:.1f}% of INR")

Reasoning Input Prepared:

1. Product Data:
{
  "product_name": "Not Available",
  "brand": "Not Available",
  "product_type": "Solid",
  "nutritional_info_per_100g": {
    "energy_kcal": 722,
    "energy_kj": "Not Available",
    "carbohydrates_g": 0,
    "sugars_g": 0,
    "added_sugars_g": 0,
    "total_fat_g": 80,
    "saturated_fat_g": 52,
    "trans_fat_g": 0,
    "sodium_mg": 836,
    "protein_g": 0.5,
    "fiber_g": "Not Available",
    "other_nutrients": {
      "cholesterol_mg": 180,
      "vitamin_a_mcg": 650
    }
  }
}

2. FSSAI Law (INR Baseline):
{
  "inr_baseline": {
    "energy_kcal": 2000,
    "total_fat_g": 65,
    "saturated_fat_g": 20,
    "carbohydrates_g": 300,
    "sugars_g": 50,
    "added_sugars_g": 30,
    "protein_g": 50,
    "sodium_mg": 2000,
    "fiber_g": 25
  },
  "positive_factor_threshold": {
    "protein": {
      "high_threshold_percent": 20,
      "source_threshold_percent": 10
    },
    "fiber": {
      "high_threshold_percent": 20,
      "source

In [28]:
# Use o3-mini reasoning model to calculate INR score
from openai import AzureOpenAI

# Initialize Azure OpenAI client for o3-mini
client = AzureOpenAI(
    api_key=NETAICONNECT_API_KEY,
    api_version="2024-12-01-preview",
    azure_endpoint="https://netaiconnect.netapp.com/"
)

# Create the reasoning prompt
reasoning_prompt = f"""You are an expert food nutrition analyst tasked with calculating the Indian Nutrition Rating (INR) score for a food product based on FSSAI guidelines.

**Product Information:**
- Product Type: {reasoning_input['product_data']['product_type']}
- Nutritional Information (per 100g):
  - Energy: {reasoning_input['product_data']['nutritional_info_per_100g']['energy_kcal']} kcal
  - Sugars: {reasoning_input['product_data']['nutritional_info_per_100g']['sugars_g']} g
  - Saturated Fat: {reasoning_input['product_data']['nutritional_info_per_100g']['saturated_fat_g']} g
  - Sodium: {reasoning_input['product_data']['nutritional_info_per_100g']['sodium_mg']} mg
  - Protein: {reasoning_input['product_data']['nutritional_info_per_100g']['protein_g']} g
  - Fiber: {reasoning_input['product_data']['nutritional_info_per_100g'].get('fiber_g', 'Not Available')}

**FSSAI INR Baseline Values (2000 kcal diet):**
- Energy: {reasoning_input['fssai_law']['inr_baseline']['energy_kcal']} kcal
- Sugars: {reasoning_input['fssai_law']['inr_baseline']['sugars_g']} g
- Saturated Fat: {reasoning_input['fssai_law']['inr_baseline']['saturated_fat_g']} g
- Sodium: {reasoning_input['fssai_law']['inr_baseline']['sodium_mg']} mg
- Protein: {reasoning_input['fssai_law']['inr_baseline']['protein_g']} g
- Fiber: {reasoning_input['fssai_law']['inr_baseline']['fiber_g']} g

**Nutrient Analysis (% of INR per 100g):**
- Energy: {analysis['energy_kcal']['percent_of_inr']:.1f}%
- Sugars: {analysis['sugars_g']['percent_of_inr']:.1f}%
- Saturated Fat: {analysis['saturated_fat_g']['percent_of_inr']:.1f}%
- Sodium: {analysis['sodium_mg']['percent_of_inr']:.1f}%
- Protein: {analysis['protein_g']['percent_of_inr']:.1f}%

**Positive Factor Thresholds:**
- Protein: High (≥20% INR), Source (≥10% INR)
- Fiber: High (≥20% INR), Source (≥10% INR)

**Task:**
Calculate the Indian Nutrition Rating (INR) score (0-100) for this product using the following methodology:

1. **Negative Points** (unhealthy nutrients - higher is worse):
   - Energy: Assign points based on % of INR (0-10 points)
   - Sugars: Assign points based on % of INR (0-10 points)
   - Saturated Fat: Assign points based on % of INR (0-10 points)
   - Sodium: Assign points based on % of INR (0-10 points)
   Total Negative: Max 40 points

2. **Positive Points** (healthy nutrients - higher is better):
   - Protein: Assign points based on % of INR (0-5 points)
   - Fiber: Assign points based on % of INR (0-5 points)
   Total Positive: Max 10 points

3. **Calculate Final Score:**
   - Raw Score = (Total Positive - Total Negative) + 40
   - Normalize to 0-100 scale
   - Score Interpretation:
     * 80-100: Excellent (A)
     * 60-79: Good (B)
     * 40-59: Average (C)
     * 20-39: Poor (D)
     * 0-19: Very Poor (E)

**Output Format (JSON):**
{{
    "negative_points": {{
        "energy": <points>,
        "sugars": <points>,
        "saturated_fat": <points>,
        "sodium": <points>,
        "total": <total_negative>
    }},
    "positive_points": {{
        "protein": <points>,
        "fiber": <points>,
        "total": <total_positive>
    }},
    "inr_score": <final_score_0_to_100>,
    "grade": "<A/B/C/D/E>",
    "interpretation": "<brief explanation>",
    "health_warnings": [<list of concerns if any>],
    "positive_claims": [<list of positive attributes if any>]
}}

Provide your reasoning and then output the final JSON.
"""

print("Sending request to o3-mini reasoning model...")
print("="*80)

Sending request to o3-mini reasoning model...


In [29]:
# Call o3-mini model for reasoning
response = client.chat.completions.create(
    model="o3-mini",
    messages=[
        {
            "role": "user",
            "content": reasoning_prompt
        }
    ],
    temperature=1,
    max_completion_tokens=10000
)

# Extract the response
reasoning_response = response.choices[0].message.content

print("Reasoning Response from o3-mini:")
print("="*80)
print(reasoning_response)
print("="*80)

Reasoning Response from o3-mini:
Below is the step‐by‐step reasoning we used before presenting the final JSON output.

1. Negative Points Calculation (each nutrient is capped at 10 points):
 a. Energy: 36.1% of INR → (36.1/100) × 10 = 3.61 points
 b. Sugars: 0% of INR → 0 points
 c. Saturated Fat: 260% of INR would be 26 points if uncapped; however, we cap this nutrient at 10 points.
 d. Sodium: 41.8% of INR → (41.8/100) × 10 = 4.18 points
 Total Negative Points = 3.61 + 0 + 10 + 4.18 = 17.79

2. Positive Points Calculation (each nutrient is capped at 5 points):
 a. Protein: 1.0% of INR is well below the source threshold (≥10% INR) so it does not qualify for a positive score → 0 points
 b. Fiber: Data not available, so 0 points
 Total Positive Points = 0 + 0 = 0

3. Raw Score and Normalization:
 Raw Score = (Total Positive – Total Negative) + 40 = (0 – 17.79) + 40 = 22.21
 Since the raw score range is 0–50 (where 0 represents the worst nutrition and 50 the best), we scale it to 0–100 b

In [30]:
# Parse the JSON response from the reasoning model
import re

# Extract JSON from the response
json_match = re.search(r'\{[\s\S]*\}', reasoning_response)
if json_match:
    inr_result = json.loads(json_match.group())
    
    print("\n" + "="*80)
    print("INDIAN NUTRITION RATING (INR) SCORE CALCULATED")
    print("="*80)
    
    print("\n📊 SCORE BREAKDOWN:")
    print("-"*80)
    
    print("\n❌ Negative Points (Unhealthy Nutrients):")
    for nutrient, points in inr_result["negative_points"].items():
        if nutrient != "total":
            print(f"   {nutrient.replace('_', ' ').title():20s}: {points:>5} points")
    print(f"   {'TOTAL NEGATIVE':20s}: {inr_result['negative_points']['total']:>5} points")
    
    print("\n✅ Positive Points (Healthy Nutrients):")
    for nutrient, points in inr_result["positive_points"].items():
        if nutrient != "total":
            print(f"   {nutrient.replace('_', ' ').title():20s}: {points:>5} points")
    print(f"   {'TOTAL POSITIVE':20s}: {inr_result['positive_points']['total']:>5} points")
    
    print("\n" + "="*80)
    print(f"🎯 FINAL INR SCORE: {inr_result['inr_score']}/100")
    print(f"📝 GRADE: {inr_result['grade']}")
    print(f"💡 INTERPRETATION: {inr_result['interpretation']}")
    print("="*80)
    
    if inr_result.get("health_warnings"):
        print("\n⚠️  HEALTH WARNINGS:")
        for warning in inr_result["health_warnings"]:
            print(f"   - {warning}")
    
    if inr_result.get("positive_claims"):
        print("\n✨ POSITIVE ATTRIBUTES:")
        for claim in inr_result["positive_claims"]:
            print(f"   - {claim}")
    
    print("\n" + "="*80)
    
    # Store result for next steps
    inr_score_result = inr_result
else:
    print("❌ Could not extract JSON from response")
    inr_score_result = None


INDIAN NUTRITION RATING (INR) SCORE CALCULATED

📊 SCORE BREAKDOWN:
--------------------------------------------------------------------------------

❌ Negative Points (Unhealthy Nutrients):
   Energy              :  3.61 points
   Sugars              :     0 points
   Saturated Fat       :    10 points
   Sodium              :  4.18 points
   TOTAL NEGATIVE      : 17.79 points

✅ Positive Points (Healthy Nutrients):
   Protein             :     0 points
   Fiber               :     0 points
   TOTAL POSITIVE      :     0 points

🎯 FINAL INR SCORE: 44.42/100
📝 GRADE: C
💡 INTERPRETATION: Average nutritional quality; the extremely high saturated fat level drives down the score despite low sugars.

⚠️  HEALTH WARNINGS:
   - High saturated fat content



In [31]:
# Summary of Reasoning Step
print("\n" + "="*80)
print("STEP 3 COMPLETED: REASONING & INR SCORE CALCULATION")
print("="*80)

print("\n✓ Successfully used o3-mini reasoning model to:")
print("  1. Analyze product nutritional data")
print("  2. Compare against FSSAI INR baseline values")
print("  3. Calculate negative points for unhealthy nutrients")
print("  4. Calculate positive points for healthy nutrients")
print("  5. Compute final INR score (0-100 scale)")
print("  6. Assign grade (A/B/C/D/E)")
print("  7. Identify health warnings and positive claims")

print("\n📋 Key Findings:")
print(f"  • Product Type: {product_category}")
print(f"  • INR Score: {inr_score_result['inr_score']:.2f}/100")
print(f"  • Grade: {inr_score_result['grade']}")
print(f"  • Major Concern: {inr_score_result['health_warnings'][0] if inr_score_result.get('health_warnings') else 'None'}")

print("\n" + "="*80)
print("Next Step: Generate nutrition label and visualizations")
print("="*80)


STEP 3 COMPLETED: REASONING & INR SCORE CALCULATION

✓ Successfully used o3-mini reasoning model to:
  1. Analyze product nutritional data
  2. Compare against FSSAI INR baseline values
  3. Calculate negative points for unhealthy nutrients
  4. Calculate positive points for healthy nutrients
  5. Compute final INR score (0-100 scale)
  6. Assign grade (A/B/C/D/E)
  7. Identify health warnings and positive claims

📋 Key Findings:
  • Product Type: Solid
  • INR Score: 44.42/100
  • Grade: C
  • Major Concern: High saturated fat content

Next Step: Generate nutrition label and visualizations
